<a href="https://colab.research.google.com/github/L3opoldCamus/Hold-em-bot/blob/main/Hold'em_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install git+https://github.com/worldveil/deuces

  Cloning https://github.com/worldveil/deuces to /tmp/pip-req-build-x_88sqq6
  Running command git clone --filter=blob:none --quiet https://github.com/worldveil/deuces /tmp/pip-req-build-x_88sqq6
  Resolved https://github.com/worldveil/deuces to commit f42e2490187de2c4697ddb8df85e8613147e0a86
  Preparing metadata (setup.py) ... done
  Created wheel for deuces: filename=deuces-0.2.1-py3-none-any.whl size=13532 sha256=f2731bb36574be7db5974e9e4660d515d906271f8b3017301e3770342855fc0a
  Stored in directory: /tmp/pip-ephem-wheel-cache-tjve8k1l/wheels/19/d4/9b/83fdddfbf1c963852d34e7520899428ca4c1133f4c239ce869
Successfully built deuces


# Main Game


## Main Function
- Initialize Game
- call `play_game()`

In [ ]:
def main(cfr_trainer):
    player_chips = 1000.0
    bot_chips = 1000.0
    global BIG_BLIND
    BIG_BLIND = 10.0

    print("--- 德州撲克遊戲開始 ---")
    print(f"你的起始籌碼: {player_chips/BIG_BLIND} BB")
    print(f"Bot 的起始籌碼: {bot_chips/BIG_BLIND} BB")

    hand_num = 1
    while player_chips > 0 and bot_chips > 0 and hand_num <= 100: # Play up to 100 hands or until one player is busted
        print(f"\n====== 第 {hand_num} 局 ====== ")
        print(f"玩家: {player_chips/BIG_BLIND} BB")
        print(f"Bot: {bot_chips/BIG_BLIND} BB")

        # Dealer alternates each hand. 0 for player, 1 for bot.
        # Simplified: Player is dealer on odd hands (hand_num % 2 != 0), Bot on even hands.
        dealer_position = 0 if hand_num % 2 != 0 else 1

        player_chips, bot_chips, winner = play_game(player_chips, bot_chips, dealer_position, cfr_trainer)

        print(f"------ 第 {hand_num} 局結束 ------")
        print(f"玩家: {player_chips/BIG_BLIND} BB")
        print(f"Bot: {bot_chips/BIG_BLIND} BB")
        hand_num += 1

    print("\n--- 遊戲結束 ---")
    if player_chips <= 0:
        print("玩家已用盡。Bot 獲勝！")
    elif bot_chips <= 0:
        print("Bot已用盡。你獲勝！")
    else:
        print("達到最大局數。")
        if player_chips > bot_chips:
            print("你以更多籌碼獲勝！")
        elif bot_chips > player_chips:
            print("Bot 以更多籌碼獲勝！")
        else:
            print("平局！")

## play_game
- 發牌：為玩家和Bot發手牌，並依序發出公共牌（翻牌、轉牌、河牌）。
- 盲注處理：根據莊家位置設定小盲注和大盲注。
- 下注回合管理：循環進行翻牌前、翻牌、轉牌、河牌四個下注回合，處理玩家和Bot的行動（過牌、下注、加注、跟注、棄牌、全下）。
- 全下攤牌：若遊戲中觸發全下（All-in）條件，將自動跳過剩餘下注回合，直接發出所有公共牌並進入攤牌。
- 遊戲勝負判定：在攤牌階段比較玩家和Bot的牌型，決定底池歸屬，並更新各自的籌碼。

In [ ]:
def play_game(player_chips_start, bot_chips_start, dealer_position, cfr_trainer):
    global BIG_BLIND

    # Initialize for the hand
    deck = Deck()
    board = []
    pot = 0
    hand_winner = None
    all_in_showdown_triggered = False # New flag to control skipping betting rounds

    # Deal hole cards
    player_hole_cards = deck.draw(2)
    bot_hole_cards = deck.draw(2)

    # Display player's hole cards
    print("你的手牌:", Card.print_pretty_cards(player_hole_cards))

    # Determine blinds
    sb_player_is_human = False
    bb_player_is_human = False

    if dealer_position == 0: # Player is dealer, bot is SB, player is BB
        # In 2-player, dealer is SB. Let's make dealer = SB, non-dealer = BB for simplicity
        # So if player is dealer (position 0), player is SB.
        sb_player_is_human = True
        bb_player_is_human = False # Bot is BB
        print("你 (Player) 是 SB，Bot 是 BB。")
    else: # Bot is dealer, player is BB
        sb_player_is_human = False # Bot is SB
        bb_player_is_human = True
        print("Bot 是 SB，你 (Player) 是 BB。")

    # --- Pre-flop Betting Round ---
    print("\n--- 翻牌前下注 ---")
    player_chips_after_preflop, bot_chips_after_preflop, pot_after_preflop, winner_preflop, is_all_in_showdown_preflop = run_betting_round(
        player_chips_start, bot_chips_start, pot, player_hole_cards, bot_hole_cards, [], True, sb_player_is_human, bb_player_is_human, BIG_BLIND, cfr_trainer)

    if winner_preflop:
        return player_chips_after_preflop, bot_chips_after_preflop, winner_preflop

    player_chips_current = player_chips_after_preflop
    bot_chips_current = bot_chips_after_preflop
    pot_current = pot_after_preflop
    all_in_showdown_triggered = is_all_in_showdown_preflop

    # --- Flop ---
    if not all_in_showdown_triggered:
        board += deck.draw(3) # Deal 3 community cards for the flop
        print("\n--- 翻牌 (Flop) ---")
        print("公共牌:", Card.print_pretty_cards(board))
        player_chips_after_flop, bot_chips_after_flop, pot_after_flop, winner_flop, is_all_in_showdown_flop = run_betting_round(
            player_chips_current, bot_chips_current, pot_current, player_hole_cards, bot_hole_cards, board, False, sb_player_is_human, bb_player_is_human, BIG_BLIND, cfr_trainer)

        if winner_flop:
            return player_chips_after_flop, bot_chips_after_flop, winner_flop

        player_chips_current = player_chips_after_flop
        bot_chips_current = bot_chips_after_flop
        pot_current = pot_after_flop
        all_in_showdown_triggered = all_in_showdown_triggered or is_all_in_showdown_flop

    # --- Turn ---
    if not all_in_showdown_triggered:
        board.append(deck.draw(1)[0]) # Simplified card dealing

        print("\n--- 轉牌 (Turn) ---")
        print("公共牌:", Card.print_pretty_cards(board))
        player_chips_after_turn, bot_chips_after_turn, pot_after_turn, winner_turn, is_all_in_showdown_turn = run_betting_round(
            player_chips_current, bot_chips_current, pot_current, player_hole_cards, bot_hole_cards, board, False, sb_player_is_human, bb_player_is_human, BIG_BLIND, cfr_trainer)

        if winner_turn:
            return player_chips_after_turn, bot_chips_after_turn, winner_turn

        player_chips_current = player_chips_after_turn
        bot_chips_current = bot_chips_after_turn
        pot_current = pot_after_turn
        all_in_showdown_triggered = all_in_showdown_triggered or is_all_in_showdown_turn

    # --- River ---
    if not all_in_showdown_triggered:
        board.append(deck.draw(1)[0]) # Simplified card dealing

        print("\n--- 河牌 (River) ---")
        print("公共牌:", Card.print_pretty_cards(board))
        player_chips_after_river, bot_chips_after_river, pot_after_river, winner_river, is_all_in_showdown_river = run_betting_round(
            player_chips_current, bot_chips_current, pot_current, player_hole_cards, bot_hole_cards, board, False, sb_player_is_human, bb_player_is_human, BIG_BLIND, cfr_trainer)

        if winner_river:
            return player_chips_after_river, bot_chips_after_river, winner_river

        player_chips_current = player_chips_after_river
        bot_chips_current = bot_chips_after_river
        pot_current = pot_after_river
        # No need to update all_in_showdown_triggered after river, as it's the last betting round.
        # The showdown will happen regardless.

    # --- Handle All-in Showdown: Deal remaining cards if triggered ---
    if all_in_showdown_triggered:
        while len(board) < 5:
            board.append(deck.draw(1)[0]) # Simplified card dealing
        print("--- 最終公共牌:", Card.print_pretty_cards(board), "---")


    # --- Showdown ---
    print("\n--- 攤牌 (Showdown) ---")
    evaluator = Evaluator()

    player_hand_rank = evaluator.evaluate(board, player_hole_cards)
    bot_hand_rank = evaluator.evaluate(board, bot_hole_cards)

    player_hand_str = Card.print_pretty_cards(player_hole_cards)
    bot_hand_str = Card.print_pretty_cards(bot_hole_cards)

    print(f"你的手牌: {player_hand_str}, {evaluator.get_rank_class(player_hand_rank)} ({evaluator.class_to_string(evaluator.get_rank_class(player_hand_rank))})")
    print(f"Bot 的手牌: {bot_hand_str}, {evaluator.get_rank_class(bot_hand_rank)} ({evaluator.class_to_string(evaluator.get_rank_class(bot_hand_rank))})")

    # Determine winner
    if player_hand_rank < bot_hand_rank: # Lower rank is better
        print("你贏得底池！")
        player_chips_current += pot_current
        hand_winner = "player"
    elif bot_hand_rank < player_hand_rank:
        print("Bot 贏得底池！")
        bot_chips_current += pot_current
        hand_winner = "bot"
    else:
        print("平手，底池平分！")
        player_chips_current += pot_current / 2
        bot_chips_current += pot_current / 2
        hand_winner = "tie"

    return player_chips_current, bot_chips_current, hand_winner

##Get Player Bet Action


In [ ]:
def get_player_bet_action(player_committed_this_round, highest_bet_this_round, player_chips, BIG_BLIND):
    player_amount_to_match = highest_bet_this_round - player_committed_this_round

    # --- NEW LOGIC: Handle cases where player has no chips left ---
    if player_chips <= 0:
        if player_amount_to_match > 0: # Facing a bet with no chips
            # print("你沒有籌碼可以跟注或下注，強制棄牌。")
            return ("fold", 0)
        else: # No bet faced, but no chips. Effectively a forced check.
            # print("你沒有籌碼可以下注，強制check。")
            return ("check", 0)
    # --- END NEW LOGIC ---

    if player_amount_to_match == 0:
        # Player can check or bet (not facing a bet)
        while True:
            action = input(f"你要 [check(c) / bet(b)]? ").lower()
            if action == "c":
                return ("check", 0)
            elif action == "b":
                while True:
                    try:
                        bet_bb_units = float(input(f"下注多少大盲？ (當前最高注碼為 {highest_bet_this_round / BIG_BLIND} 大盲，你已投入 {player_committed_this_round / BIG_BLIND} 大盲))"))
                        player_new_total_bet = bet_bb_units * BIG_BLIND

                        # A bet must be at least BIG_BLIND if it's the first bet
                        if player_new_total_bet <= 0:
                            print("下注金額必須大於0。請重新輸入。")
                        elif player_new_total_bet < BIG_BLIND and highest_bet_this_round == 0: # First bet must be at least 1 BB
                            print(f"首次下注金額必須至少為 {BIG_BLIND / BIG_BLIND} 大盲。請重新輸入。")
                        elif (player_new_total_bet - player_committed_this_round) > player_chips:
                            print(f"籌碼不足。你只有 {player_chips / BIG_BLIND} 大盲。請重新輸入。")
                        else:
                            return ("bet", player_new_total_bet)
                    except ValueError:
                        print("請輸入有效的數字。")
            else:
                print("無效的輸入。請輸入 'c' 或 'b'。")
    else:
        # Player is facing a bet
        print(f"你需要跟注 {player_amount_to_match / BIG_BLIND} 大盲.")
        while True:
            # Check if player can afford to call
            can_afford_full_call = (player_amount_to_match <= player_chips)

            # Options presented to the player
            if can_afford_full_call:
                action_prompt = f"你要 [call / raise / fold / all-in]? (輸入 'c' 跟注, 'r' 加注, 'f' 棄牌, 'a' 全下) "
            else:
                action_prompt = f"你籌碼不足以跟注。你要 [all-in / fold]? (輸入 'a' 全下 {player_chips / BIG_BLIND} 大盲, 'f' 棄牌) "

            action = input(action_prompt).lower()

            if action == "f":
                return ("fold", 0)
            elif action == "c":
                if can_afford_full_call: # Allow 'c' if can afford full call
                    return ("call", highest_bet_this_round)
                else:
                    # If cannot afford full call, and player tries 'c', treat as all-in
                    if player_chips > 0:
                        print(f"你沒有足夠的籌碼來跟注 {player_amount_to_match / BIG_BLIND} 大盲。你只有 {player_chips / BIG_BLIND} 大盲。將自動視為全下。")
                        return ("call", player_committed_this_round + player_chips) # Effectively an all-in call
                    else:
                        print("你沒有籌碼可以全下了。請輸入 'f' 棄牌。")
                        return ("fold", 0) # Force fold to break infinite loop
            elif action == "a": # Explicit all-in
                if player_chips > 0:
                    all_in_total_bet = player_committed_this_round + player_chips
                    # print(f"DEBUG (get_player_bet_action): player_committed_this_round={player_committed_this_round/BIG_BLIND}BB, player_chips={player_chips/BIG_BLIND}BB, highest_bet_this_round={highest_bet_this_round/BIG_BLIND}BB, all_in_total_bet={all_in_total_bet/BIG_BLIND}BB")
                    # print(f"DEBUG (get_player_bet_action): Comparing all_in_total_bet ({all_in_total_bet}) vs highest_bet_this_round ({highest_bet_this_round})")
                    if all_in_total_bet > highest_bet_this_round:
                        # print(f"DEBUG (get_player_bet_action): Returning ('raise', {all_in_total_bet}) for ALL-IN as RAISE")
                        return ("raise", all_in_total_bet) # All-in is a raise
                    else:
                        # print(f"DEBUG (get_player_bet_action): Returning ('call', {all_in_total_bet}) for ALL-IN as CALL")
                        return ("call", all_in_total_bet) # All-in is a call (or short-call)
                else:
                    return ("fold", 0) # Force fold to break infinite loop
            elif action == "r":
                if can_afford_full_call: # Only allow 'r' if can afford full call
                    while True:
                        try:
                            new_total_bet_bb_units = float(input(f"加注到多少大盲 (當前最高注碼為 {highest_bet_this_round / BIG_BLIND} 大盲，你已投入 {player_committed_this_round / BIG_BLIND} 大盲)？"))
                            player_new_total_bet = new_total_bet_bb_units * BIG_BLIND

                            min_valid_total_bet_for_raise = highest_bet_this_round + BIG_BLIND

                            if player_new_total_bet <= highest_bet_this_round:
                                print(f"加注金額 ({player_new_total_bet / BIG_BLIND} 大盲) 必須大於當前最高注碼 ({highest_bet_this_round / BIG_BLIND} 大盲)。請重新輸入。")
                            elif player_new_total_bet < min_valid_total_bet_for_raise:
                                print(f"合法加注必須至少加注到 {min_valid_total_bet_for_raise / BIG_BLIND} 大盲。請重新輸入。")
                            elif (player_new_total_bet - player_committed_this_round) > player_chips:
                                 print(f"你沒有足夠的籌碼來加注到 {player_new_total_bet / BIG_BLIND} 大盲。你只有 {player_chips / BIG_BLIND} 大盲。請重新輸入。張。")
                            else:
                                return ("raise", player_new_total_bet)
                        except ValueError:
                            print("請輸入有效的數字。")
                else:
                    print("籌碼不足以加注，請選擇其他動作。")
            else:
                print("無效的輸入。請輸入有效的動作。")

##Run betting round
incorporating all the betting logic, player input, and validation rules.`


In [ ]:
def run_betting_round(player_chips_current, bot_chips_current, pot_current, player_hole_cards, bot_hole_cards, board, is_preflop, sb_player_is_human, bb_player_is_human, BIG_BLIND, cfr_trainer):
    """
    Manages a single betting street (e.g., preflop, flop, turn, river) for two players.

    Args:
        player_chips_current (float): Current chip count for the player at the start of this betting round.
        bot_chips_current (float): Current chip count for the bot at the start of this betting round.
        pot_current (float): Current total pot from previous rounds.
        player_hole_cards (list): Player's private cards.
        bot_hole_cards (list): Bot's private cards.
        board (list): Community cards.
        is_preflop (bool): True if this is the pre-flop betting round.
        sb_player_is_human (bool): True if the human player is in the small blind position.
        bb_player_is_human (bool): True if the human player is in the big blind position.
        BIG_BLIND (int): The value of the big blind.
        cfr_trainer: The trained CFRTrainer instance for bot decisions.

    Returns:
        tuple: (updated_player_chips, updated_bot_chips, updated_pot, winner_found_in_round, is_all_in_showdown)
               winner_found_in_round is 'player', 'bot', or None if no fold occurred.
               is_all_in_showdown is True if, after a stable round with no folds, at least one player has committed all their chips.
    """

    # Debug print: Initial state of the round
    print(f"\nDEBUG (run_betting_round): Starting round. Player Chips: {player_chips_current/BIG_BLIND:.2f} BB, Bot Chips: {bot_chips_current/BIG_BLIND:.2f} BB, Pot: {pot_current/BIG_BLIND:.2f} BB, Is Preflop: {is_preflop}")

    # Initialize variables for the current betting round
    player_committed_this_round = 0
    bot_committed_this_round = 0
    highest_bet_this_round = 0

    current_player_chips_in_round = player_chips_current # Chips remaining after committing to this round
    current_bot_chips_in_round = bot_chips_current # Chips remaining after committing to this round
    current_pot = pot_current

    winner_found_in_round = None # Stores 'player', 'bot', or None if no fold
    is_all_in_showdown = False # New flag for all-in showdown

    # Handle blinds for pre-flop
    if is_preflop:
        print("\n=== Pre-flop Blinds ===")
        sb_amount = 0.5 * BIG_BLIND
        bb_amount = BIG_BLIND

        if sb_player_is_human:
            # Human is SB, Bot is BB
            # Ensure players have enough chips for blinds, if not, they are all-in for the blind amount
            player_blind_amount = min(sb_amount, current_player_chips_in_round)
            bot_blind_amount = min(bb_amount, current_bot_chips_in_round)

            player_committed_this_round = player_blind_amount
            bot_committed_this_round = bot_blind_amount

            current_player_chips_in_round -= player_blind_amount
            current_bot_chips_in_round -= bot_blind_amount
            current_pot += (player_blind_amount + bot_blind_amount)
            print(f"你 (SB) 放入了 {player_blind_amount / BIG_BLIND} 大盲。Bot (BB) 放入了 {bot_blind_amount / BIG_BLIND} 大盲。")
        else:
            # Bot is SB, Human is BB
            # Ensure players have enough chips for blinds, if not, they are all-in for the blind amount
            bot_blind_amount = min(sb_amount, current_bot_chips_in_round)
            player_blind_amount = min(bb_amount, current_player_chips_in_round)

            bot_committed_this_round = bot_blind_amount
            player_committed_this_round = player_blind_amount

            current_bot_chips_in_round -= bot_blind_amount
            current_player_chips_in_round -= player_blind_amount
            current_pot += (bot_blind_amount + player_blind_amount)
            print(f"Bot (SB) 放入了 {bot_blind_amount / BIG_BLIND} 大盲。你 (BB) 放入了 {player_blind_amount / BIG_BLIND} 大盲。")

        # The highest bet is always the max committed by either player, accounting for potential all-in blinds
        highest_bet_this_round = max(player_committed_this_round, bot_committed_this_round)
        print(f"當前底池: {current_pot / BIG_BLIND} 大盲")

        # Debug print: State after blinds
        print(f"DEBUG (run_betting_round): After blinds. Player Chips Left: {current_player_chips_in_round/BIG_BLIND:.2f} BB, Bot Chips Left: {current_bot_chips_in_round/BIG_BLIND:.2f} BB, Current Pot: {current_pot/BIG_BLIND:.2f} BB")
        print(f"DEBUG (run_betting_round): Player Committed: {player_committed_this_round/BIG_BLIND:.2f} BB, Bot Committed: {bot_committed_this_round/BIG_BLIND:.2f} BB, Highest Bet: {highest_bet_this_round/BIG_BLIND:.2f} BB")

    # Determine who acts first in the actual betting for this street
    # Pre-flop: Player in SB position acts first, responding to BB (i.e., player to the left of BB)
    # Post-flop: Player in SB position acts first (with option to check/bet).
    # `player_to_act` is True if human is currently to act, False if bot.
    player_to_act = sb_player_is_human

    # Flags to track if players have completed their action sequence for this round's highest bet.
    # These reset if a new raise occurs. Should be initialized once per betting round.
    player_has_acted_this_sequence = False
    bot_has_acted_this_sequence = False
    last_action_was_raise_or_bet = False # True if the last action was an aggressive bet/raise

    round_is_stable = False

    while not round_is_stable and winner_found_in_round is None:
        # Ensure flags are reset only when a new betting sequence starts (after a raise)
        # not at the beginning of every loop iteration if no raise occurred.

        if player_to_act: # Human player's turn
            print(f"\n玩家: {current_player_chips_in_round / BIG_BLIND} 大盲, Bot 籌碼: {current_bot_chips_in_round / BIG_BLIND} 大盲")
            print(f"你已投入本輪: {player_committed_this_round / BIG_BLIND} 大盲, Bot 已投入本輪: {bot_committed_this_round / BIG_BLIND} 大盲")
            print(f"當前底池: {current_pot / BIG_BLIND} 大盲, 最高注碼: {highest_bet_this_round / BIG_BLIND} 大盲")

            # Call get_player_bet_action without is_first_action_of_sequence
            action_type, action_value = get_player_bet_action(player_committed_this_round, highest_bet_this_round, current_player_chips_in_round, BIG_BLIND)
            # print(f"DEBUG (run_betting_round, PLAYER): After get_player_bet_action: action_type='{action_type}', action_value={action_value/BIG_BLIND}BB") # REMOVED DEBUG LINE

            if action_type == "fold":
                print("你棄牌。Bot 勝！")
                winner_found_in_round = "bot"
                current_bot_chips_in_round += current_pot # Bot wins the entire pot
                break
            elif action_type == "call":
                amount_needed_to_call = highest_bet_this_round - player_committed_this_round
                amount_to_add = min(amount_needed_to_call, current_player_chips_in_round)
                current_player_chips_in_round -= amount_to_add
                current_pot += amount_to_add
                player_committed_this_round += amount_to_add # Update committed amount
                print(f"你跟注 {amount_to_add / BIG_BLIND} 大盲。你目前投入本輪: {player_committed_this_round / BIG_BLIND} 大盲。")
                player_has_acted_this_sequence = True
                last_action_was_raise_or_bet = False
            elif action_type == "raise" or action_type == "bet":
                amount_to_add = action_value - player_committed_this_round
                current_player_chips_in_round -= amount_to_add
                current_pot += amount_to_add
                player_committed_this_round = action_value
                highest_bet_this_round = action_value
                print(f"你 {action_type} 到 {action_value / BIG_BLIND} 大盲。你目前投入本輪: {player_committed_this_round / BIG_BLIND} 大盲。")
                last_action_was_raise_or_bet = True
                player_has_acted_this_sequence = True
                bot_has_acted_this_sequence = False # Bot needs to respond to the new raise
            elif action_type == "check":
                print("你選擇 check。")
                player_has_acted_this_sequence = True
                last_action_was_raise_or_bet = False

        else: # Bot's turn
            print("\nBot 的回合...")
            # Bot needs to know its current commitment, and the highest bet level to decide.
            bot_action, bot_final_total_bet_level, eq = bot_decide(bot_hole_cards, board, current_pot, bot_committed_this_round, highest_bet_this_round, current_bot_chips_in_round, BIG_BLIND, cfr_trainer)

            if bot_action == "fold":
                print("Bot 棄牌。你贏了！")
                winner_found_in_round = "player"
                current_player_chips_in_round += current_pot # Player wins the entire pot
                break
            elif bot_action == "call":
                amount_to_add = highest_bet_this_round - bot_committed_this_round
                # Fix for bot's call action: cap amount_to_add at current_bot_chips_in_round
                amount_to_add = min(amount_to_add, current_bot_chips_in_round) if amount_to_add > 0 else 0

                current_bot_chips_in_round -= amount_to_add
                current_pot += amount_to_add
                bot_committed_this_round += amount_to_add # Update committed amount based on actual added

                if amount_to_add == 0 and highest_bet_this_round == bot_committed_this_round: # If bot is 'calling' 0, it's a check (when no bet is faced, and bot is not committed more)
                    print("Bot 選擇 check。")
                else:
                    print(f"Bot 跟注 {amount_to_add / BIG_BLIND} 大盲。Bot 目前投入本輪: {bot_committed_this_round / BIG_BLIND} 大盲。(勝率估計={eq:.2f})")
                bot_has_acted_this_sequence = True
                last_action_was_raise_or_bet = False
            elif bot_action == "raise" or bot_action == "bet": # Combined handling for bet and raise
                amount_to_add = bot_final_total_bet_level - bot_committed_this_round
                current_bot_chips_in_round -= amount_to_add
                current_pot += amount_to_add
                bot_committed_this_round = bot_final_total_bet_level
                highest_bet_this_round = bot_final_total_bet_level # Bot's action sets new highest bet

                action_type_chinese = "下注" if bot_action == "bet" else "加注" # Determine Chinese word based on bot_action

                print(f"Bot {action_type_chinese} 到 {bot_final_total_bet_level / BIG_BLIND} 大盲。Bot 目前投入本輪: {bot_committed_this_round / BIG_BLIND} 大盲。(勝率估計={eq:.2f})")
                last_action_was_raise_or_bet = True
                bot_has_acted_this_sequence = True
                player_has_acted_this_sequence = False # Player needs to respond to the new action

        # Switch active player for next iteration
        player_to_act = not player_to_act

        # Check for round stability
        # Round is stable if:
        # 1. Both players have committed the same amount to the highest bet this round.
        # 2. Both players have acted since the last aggressive action (if any).
        is_betting_matched = (player_committed_this_round == highest_bet_this_round) and \
                             (bot_committed_this_round == highest_bet_this_round)

        if is_betting_matched and player_has_acted_this_sequence and bot_has_acted_this_sequence: # This condition implies check-check or call-bet-call/raise-call
            # If everyone has matched the highest bet and both have acted since the last aggressive action, or if it was check-check, then the round is stable.
            round_is_stable = True
            if winner_found_in_round is None: # Only check for all-in showdown if no one folded
                # Debug print: Current chip status before all-in check
                print(f"DEBUG (run_betting_round): Checking for all-in showdown. Player Chips Left: {current_player_chips_in_round/BIG_BLIND:.2f} BB, Bot Chips Left: {current_bot_chips_in_round/BIG_BLIND:.2f} BB")
                # An all-in showdown occurs if the round is stable AND at least one player is out of chips
                if current_player_chips_in_round <= 0 or current_bot_chips_in_round <= 0:
                    is_all_in_showdown = True
                    print("--- 全下攤牌條件滿足！所有後續下注回合將被跳過。 ---")
            print(f"\n--- 下注回合結束。底池: {current_pot / BIG_BLIND} 大盲 ---")

    # The actual chip updates for the hand are handled outside this function.
    # This function returns the remaining chips for the *current round*

    # Debug print: Final state of the round
    print(f"DEBUG (run_betting_round): Round finished. Player Chips Left (after round actions): {current_player_chips_in_round/BIG_BLIND:.2f} BB, Bot Chips Left: {current_bot_chips_in_round/BIG_BLIND:.2f} BB, Pot in round: {current_pot/BIG_BLIND:.2f} BB")

    return current_player_chips_in_round, current_bot_chips_in_round, current_pot, winner_found_in_round, is_all_in_showdown

# CFR+ Trainer
1.  **Define `CFRTrainer` class**: Implement `_get_possible_opponent_hands` to generate all legal 2-card combinations for the opponent, and `_calculate_showdown_equity_mc` to run Monte Carlo simulations for showdown equity, sampling opponent hands from the generated possibilities.
2.  **Implement `bot_decide` function**:
    *   This function will evaluate potential actions: 'fold', 'check', 'call', 'bet' (half-pot), 'raise', and 'all-in'.
    *   For each action, it will calculate an "expected final chip stack" as its Expected Value (EV) by considering:
        *   **Fold**: Bot loses its committed chips for the current round.
        *   **Check/Call**: Assume the game proceeds to showdown. EV is calculated based on `_calculate_showdown_equity_mc` and the resulting pot size.
        *   **Bet/Raise/All-in**: Use a simple heuristic for opponent's response: the opponent folds with a certain probability (e.g., 30% for bets/raises, 50% for all-ins) or calls otherwise. If the opponent calls, the game is assumed to go to showdown, and `_calculate_showdown_equity_mc` is used.
    *   The bot will choose the action that maximizes its expected final chip stack.
3.  **Integrate `bot_decide`**: Ensure `run_betting_round` correctly calls this new `bot_decide` function.
4.  **Test**: A game will be run implicitly later to verify the bot's behavior.

Let's start by defining the `CFRTrainer` class and the `bot_decide` function.


In [ ]:
import numpy as np
import random
from deuces import Card, Evaluator, Deck
from collections import defaultdict

# --- 全局常數設定 ---
# 為了簡化邏輯，假設底池已經有一些錢 (例如 100)，雙方剩餘籌碼 (Stack) 為 200
STARTING_POT = 100
STACK_SIZE = 200
BET_SIZE = 50       # 限制注模式：固定下注額
RAISE_SIZE = 100    # 限制注模式：固定加注額

# 行動空間
ACTIONS = ['fold', 'check', 'call', 'bet', 'raise']
NUM_ACTIONS = len(ACTIONS)

class CFRNode:
    def __init__(self, num_actions):
        self.regret_sum = np.zeros(num_actions)
        self.strategy_sum = np.zeros(num_actions)
        self.visits = 0

    def get_strategy(self, realization_weight):
        """
        Regret Matching: 計算當前混合策略
        """
        strategy = np.maximum(self.regret_sum, 0)
        normalizing_sum = np.sum(strategy)

        if normalizing_sum > 0:
            strategy /= normalizing_sum
        else:
            strategy = np.ones(NUM_ACTIONS) / NUM_ACTIONS

        self.strategy_sum += strategy * realization_weight
        return strategy

    def get_average_strategy(self):
        """
        獲取最終訓練好的納許均衡策略
        """
        normalizing_sum = np.sum(self.strategy_sum)
        if normalizing_sum > 0:
            return self.strategy_sum / normalizing_sum
        else:
            return np.ones(NUM_ACTIONS) / NUM_ACTIONS

class SimpleTexasCFR:
    def __init__(self):
        self.evaluator = Evaluator()
        self.nodes = {} # Map: info_set_key -> CFRNode

    def get_node(self, info_set_key):
        if info_set_key not in self.nodes:
            self.nodes[info_set_key] = CFRNode(NUM_ACTIONS)
        return self.nodes[info_set_key]

    def _generate_key(self, hand, board, history):
        """
        關鍵：生成狀態的唯一標識符 (Abstraction)
        你應該在這裡加入你的「卡牌抽象化」邏輯。
        目前範例：只使用手牌 Rank + 行動歷史
        """
        # 簡單範例：將手牌轉為字串 (例如 "AhKh")
        # 實際應用中，請用你的 Bucket ID 替換這裡
        hand_str = Card.ints_to_string(hand).replace(" ", "")
        board_str = "River" # 簡化：假設只在 River
        history_str = "/".join(history)
        return f"{hand_str}|{board_str}|{history_str}"

    def _calculate_pot_contribution(self, history):
        """
        解析歷史紀錄，計算雙方投入的籌碼
        P0 (先手), P1 (後手)
        """
        p0_contribution = 0
        p1_contribution = 0

        # 模擬一個簡單的狀態機來計算籌碼
        # 假設：bet = 50, raise = 100 (固定限注)
        current_bet = 0

        for i, action in enumerate(history):
            is_p0 = (i % 2 == 0)

            if action == 'bet':
                amount = BET_SIZE
                current_bet = amount
                if is_p0: p0_contribution = amount
                else: p1_contribution = amount

            elif action == 'raise':
                # Raise 假設是蓋過之前的下注並加碼
                amount = RAISE_SIZE
                current_bet = amount
                if is_p0: p0_contribution = amount
                else: p1_contribution = amount

            elif action == 'call':
                # Call 補齊到當前最高注額
                if is_p0: p0_contribution = p1_contribution
                else: p1_contribution = p0_contribution

            # Fold 和 Check 不增加籌碼投入

        return p0_contribution, p1_contribution

    def _is_terminal(self, history):
        """
        判斷是否到達終止狀態
        """
        if not history:
            return False

        last_action = history[-1]

        # 1. 有人棄牌 -> 結束
        if last_action == 'fold':
            return True

        # 2. 攤牌 (Showdown) 情況
        # Case A: check -> check
        if len(history) >= 2 and history[-1] == 'check' and history[-2] == 'check':
            return True
        # Case B: bet -> call 或 raise -> call
        if last_action == 'call':
            return True

        # 簡單限制：防止樹過深 (例如最多 3 次行動)
        if len(history) >= 3:
            return True

        return False

    def _get_utility(self, history, cards, board, active_player):
        """
        計算效用 (Utility)。
        注意：這裡是零和博弈。返回值是對「當前行動者 (active_player)」而言的獲利/損失。
        """
        p0_hand, p1_hand = cards[0], cards[1]
        last_action = history[-1]

        # 計算雙方投入底池的錢 (不包含起始底池)
        p0_contrib, p1_contrib = self._calculate_pot_contribution(history)
        total_pot = STARTING_POT + p0_contrib + p1_contrib

        # --- 情況 1: 對手棄牌 (Fold) ---
        if last_action == 'fold':
            # 如果是最後一個動作是 fold，代表是「active_player」棄牌了嗎？
            # 不，CFR 遞迴中，_get_utility 是在 active_player 做動作之前被呼叫嗎？
            # 通常邏輯：如果 history 最後是 fold，代表上一位玩家 (previous_player) 棄牌了。
            # 所以贏家是 active_player。

            # 贏得底池 (包含起始底池 + 對手貢獻)
            # 獲利 = 對手貢獻 + 起始底池的一半 (假設這局是平分的)
            # 簡單算法：贏家拿走整個 Pot，扣除自己投入的成本 = 淨利

            # 因為上一手棄牌，當前 active_player 獲勝
            return total_pot - (p0_contrib if active_player == 0 else p1_contrib)

        # --- 情況 2: 攤牌 (Showdown) ---
        # 使用 Deuces 評分 (分數越小越強)
        p0_score = self.evaluator.evaluate(board, p0_hand)
        p1_score = self.evaluator.evaluate(board, p1_hand)

        winner = -1 # 0: P0 wins, 1: P1 wins, 2: Tie
        if p0_score < p1_score:
            winner = 0
        elif p1_score < p0_score:
            winner = 1
        else:
            winner = 2

        my_contrib = p0_contrib if active_player == 0 else p1_contrib

        if winner == 2: # 平手
            # 拿回自己的錢 (淨利為 0)
            return 0
        elif winner == active_player:
            # 贏了：獲得底池 - 自己投入的 = 贏得對手投入的 + 起始底池
            return total_pot - my_contrib
        else:
            # 輸了：損失自己投入的錢
            return -my_contrib

    def _is_valid_action(self, action, history):
        """
        規則檢查：過濾非法行動
        """
        # 如果是第一手
        if not history:
            return action in ['check', 'bet'] # 第一手只能 check 或 bet

        last_action = history[-1]

        if action == 'fold':
            return True # 隨時可以棄牌 (除非可以 check)

        if action == 'check':
            # 只有當對手 check 或尚未下注時才能 check
            if last_action == 'check': return True
            if last_action == 'bet' or last_action == 'raise': return False
            return False

        if action == 'call':
            # 只有面對下注才能 call
            if last_action == 'bet' or last_action == 'raise': return True
            return False

        if action == 'bet':
            # 只有沒人下注時才能 bet
            if last_action == 'check' or not history: return True
            return False

        if action == 'raise':
            # 只有面對 bet 或 raise 才能 raise
            if last_action == 'bet' or last_action == 'raise': return True
            return False

        return False

    def cfr(self, cards, history, board, p0, p1, active_player):
        # 1. 終止判斷
        if self._is_terminal(history):
            return self._get_utility(history, cards, board, active_player)

        # 2. 獲取信息集節點
        my_card = cards[active_player]
        info_set_key = self._generate_key(my_card, board, history)
        node = self.get_node(info_set_key)

        # 3. 獲取當前策略
        strategy = node.get_strategy(p0 if active_player == 0 else p1)

        util = np.zeros(NUM_ACTIONS)
        node_util = 0

        # 4. 遍歷行動
        for i, action in enumerate(ACTIONS):
            if not self._is_valid_action(action, history):
                continue

            new_history = history + [action]

            # 遞迴計算效用 (注意負號：零和博弈)
            if active_player == 0:
                util[i] = -self.cfr(cards, new_history, board,
                                    p0 * strategy[i], p1, 1)
            else:
                util[i] = -self.cfr(cards, new_history, board,
                                    p0, p1 * strategy[i], 0)

            node_util += strategy[i] * util[i]

        # 5. 更新遺憾值 (Regret Update)
        counterfactual_weight = p1 if active_player == 0 else p0

        for i in range(NUM_ACTIONS):
            if self._is_valid_action(ACTIONS[i], history):
                regret = util[i] - node_util
                node.regret_sum[i] += counterfactual_weight * regret

        return node_util

    def train(self, iterations):
        """
        訓練主迴圈
        """
        deck = Deck()
        # 假設公牌是固定的 (River 局面)，或者是隨機發出
        # 為了演示收斂，我們這裡隨機發牌

        print(f"Starting training for {iterations} iterations...")
        for i in range(iterations):
            deck.shuffle()
            p0_hand = deck.draw(2)
            p1_hand = deck.draw(2)
            board = deck.draw(5)
            cards = [p0_hand, p1_hand]

            self.cfr(cards, [], board, 1.0, 1.0, 0)

            if (i+1) % 1000 == 0:
                print(f"Iteration {i+1} completed.")

# --- 執行範例 ---
if __name__ == "__main__":
    trainer = SimpleTexasCFR()
    # 訓練 10,000 次 (對於簡單模型應該足夠看到策略變化)
    trainer.train(10000)

    print("\n--- Training Complete. Inspecting Strategy ---")

    # 測試查看某個情境的策略
    # 假設我拿著 A K (Rank大約是高牌)，面對對手 check
    # 注意：這裡的 key 必須跟 _generate_key 邏輯一致
    # 這裡只是一個示意，因為實際 Key 包含具體卡牌數字

    # 列印出前 5 個訓練到的節點策略
    count = 0
    for key, node in trainer.nodes.items():
        avg_strat = node.get_average_strategy()
        # 過濾掉那些幾乎沒訪問過的節點
        if np.sum(node.strategy_sum) > 10:
            print(f"State: {key}")
            # 格式化輸出策略
            strat_dict = {action: f"{prob:.2f}" for action, prob in zip(ACTIONS, avg_strat)}
            print(f"  Strategy: {strat_dict}")
            count += 1
            if count >= 5: break

Starting training for 10000 iterations...


AttributeError: type object 'Card' has no attribute 'ints_to_string'

# Test Game Play

### Subtask:
Run a test game to verify that the bot's Monte Carlo decision-making works as expected and that the game flows correctly.


In [ ]:
# Create an instance of the CFRTrainer class
cfr_trainer_instance = CFRTrainer()

main(cfr_trainer_instance)

--- 德州撲克遊戲開始 ---
你的起始籌碼: 100.0 BB
Bot 的起始籌碼: 100.0 BB

====== 第 1 局 ====== 
玩家: 100.0 BB
Bot: 100.0 BB
  [ A ♦ ] , [ 4 ❤ ]  
你的手牌: None
你 (Player) 是 SB，Bot 是 BB。

--- 翻牌前下注 ---

DEBUG (run_betting_round): Starting round. Player Chips: 100.00 BB, Bot Chips: 100.00 BB, Pot: 0.00 BB, Is Preflop: True

=== Pre-flop Blinds ===
你 (SB) 放入了 0.5 大盲。Bot (BB) 放入了 1.0 大盲。
當前底池: 1.5 大盲
DEBUG (run_betting_round): After blinds. Player Chips Left: 99.50 BB, Bot Chips Left: 99.00 BB, Current Pot: 1.50 BB
DEBUG (run_betting_round): Player Committed: 0.50 BB, Bot Committed: 1.00 BB, Highest Bet: 1.00 BB

玩家: 99.5 大盲, Bot 籌碼: 99.0 大盲
你已投入本輪: 0.5 大盲, Bot 已投入本輪: 1.0 大盲
當前底池: 1.5 大盲, 最高注碼: 1.0 大盲
你需要跟注 0.5 大盲.
你要 [call / raise / fold / all-in]? (輸入 'c' 跟注, 'r' 加注, 'f' 棄牌, 'a' 全下) a
你 raise 到 100.0 大盲。你目前投入本輪: 100.0 大盲。

Bot 的回合...
Bot Base Showdown Win Probability: 0.34, Tie Probability: 0.05
Bot fold to 0.00 BB (total committed). Expected Final Chips: 99.00 BB. Win prob (MC): 0.00
Bot 棄牌。你贏了！
DEBUG (run_bet

KeyboardInterrupt: Interrupted by user